# Table 1
I want to include the following information regarding each dataset in my table:
- \# tasks
- \# compounds
- \# median compounds per task
- source
- raw values available?

In [75]:
import pandas as pd
import os
import statistics

import duckdb

In [50]:
input_dir = "/Users/sethhowes/Desktop/FS-Tox/multirun/2023-08-10/11-46-00"
run_dirs = os.listdir(input_dir)
run_dirs = [f"{input_dir}/{run_dir}" for run_dir in run_dirs]
assay_dirs = [f"{run_dir}/data/processed/assay" for run_dir in run_dirs]
mol_dirs = [f"{run_dir}/data/processed/feature" for run_dir in run_dirs]

In [57]:
datasets = [os.path.basename(run_dir).split("=")[1] for run_dir in run_dirs]
rows = ["# tasks", "# small molecules", "median # compounds per task", "source", "raw values available?"]
sources = ["Wellcome Sanger / Mass General", "NCI", "US EPA", "?", "Zhu 2009", "US EPA", "Ekwall 1998"]
raw_vals = ["Yes"] * 2 + ["No"] + ["Yes"] * 4

In [69]:
#datasets.remove('meic')
datasets.append('meic')
datasets

['cancerrx',
 'nci60',
 'toxcast',
 'prism',
 'acute_oral_toxicity',
 'toxval',
 'meic']

In [70]:
meta_dataset = ['in-vitro'] * 4 + ['in-vivo'] * 2 + ['human']

In [71]:
stats = pd.DataFrame(columns=datasets, index=rows)

con = duckdb.connect()

for i, (dataset, assay_dir, mol_dir, raw_val, source) in enumerate(zip(datasets, assay_dirs, mol_dirs, raw_vals, sources)):
    
    dataset_stats = []
    
    # Get # tasks
    num_tasks = len([f for f in os.listdir(assay_dir)])
    dataset_stats.append(num_tasks)
    
    # Get # compounds
    query = f"SELECT COUNT(DISTINCT canonical_smiles) FROM read_parquet('{mol_dir}/*')"
    num_compounds = con.execute(query).fetchone()[0]
    dataset_stats.append(num_compounds)
                     
    # Get median # compounds per task
    query = f"SELECT COUNT(*) AS num FROM read_parquet('{assay_dir}/*') GROUP BY assay_id"
    median_task = con.execute(query).df()
    median_task = median_task['num'].median()
    dataset_stats.append(median_task)
    
    # Add source and raw val
    dataset_stats.append(source)
    dataset_stats.append(raw_val)
    
    # Add column
    stats[dataset] = dataset_stats

In [72]:
stats.columns = pd.MultiIndex.from_arrays([meta_dataset, datasets], names=('meta', 'dataset'))

In [73]:
stats

meta                                               in-vitro                 \
dataset                                            cancerrx  nci60 toxcast   
# tasks                                                 967     67       3   
# small molecules                                       227  28519      47   
median # compounds per task                           211.0   84.0    47.0   
source                       Wellcome Sanger / Mass General    NCI  US EPA   
raw values available?                                   Yes    Yes      No   

meta                                           in-vivo                human  
dataset                      prism acute_oral_toxicity  toxval         meic  
# tasks                        125                 482       1           38  
# small molecules             3429                 657    7342         1150  
median # compounds per task  113.0               604.5  7385.0         54.5  
source                           ?            Zhu 2009  US EPA  Ekwall 1998  
raw values available?          Yes                 Yes     Yes          Yes

## Median tasks in entire benchmark

In [77]:
median_task = []

con = duckdb.connect()

for i, (dataset, assay_dir, mol_dir, raw_val, source) in enumerate(zip(datasets, assay_dirs, mol_dirs, raw_vals, sources)):
                     
    # Get median # compounds per task
    query = f"SELECT COUNT(*) AS num FROM read_parquet('{assay_dir}/*') GROUP BY assay_id"
    median_task.append(con.execute(query).df()['num'].median())

statistics.median(median_task)

113.0